## Prueba Fundamentos de Data Science
###  Gustavo Morales, G10 - 21.Oct.2019

### DESAFIO 1, HITO 1 → PRELIMINARES y ASPECTOS COMPUTACIONALES

**ENUNCIADO**

Usted trabaja para un organismo no gubernamental que está interesado en las dinámicas socioeconómicas que determinan la desigualdad de ingreso y la erradicación de la pobreza extrema, enmarcado dentro de los objetivos del desarrollo del nuevo milenio del Programa de las Naciones Unidas para el Desarrollo. Le encomiendan el desarrollo de un modelo predictivo sobre la probabilidad que un individuo presente salarios por sobre o bajo los 50.000 dólares anuales, en base a una serie de atributos sociodemográficos.

**TIPO DE PROBLEMA** → Usaré regresión logística usando aprendizaje de máquinas. Esto es porque la variable objetivo es binaria, y quiero hacer un modelo que sea capaz de predecir la ocurrencia de la misma basada en una matriz de atributos sociodemográficos.

**JUSTIFICACIÓN METRICAS**

`precision`, porque es la probabilidad de que instancias positivas sean efectivamente predichas como positivas, es decir, es una medida de la calidad del modelo;

`sensitivity` o `recall`, porque hace referencia a cuántas de las mediciones que son positivas el modelo es capaz de capturar etiquetándolas como positivas, es decir, es una medida de la completitud del modelo;

`accuracy`, porque es la fracción de predicciones correctas entre el número total de registros, es decir, es una medida intuitiva del poder de predicción del modelo;

`F1-score`, porque combina de manera razonable `precision` y `sensitivity`, es decir, es una medida de qué tan robusto es el modelo como un todo; y

`AUC`, porque el área bajo la curva ROC es una buena medida de cuán bien se pueden discriminar los casos positivos de los negativos, reduciendo los errores Tipo I y II.

Finalmente mostraré la `confusion matrix`, para poder visualizar las posibles correlaciones entre los atributos del modelo.

**JUSTIFICACIÓN DE LIBRERIAS**

`pandas`, `numpy`: para poder manejar los datos, transformarlos y prepararlos para el modelamiento.

`matplotlib.pyplot`, `seaborn`: para poder realizar gráficos pertinentes a la visualización de los datos y de los resultados.

`sklearn`: para poder aplicar las técnicas de modelamiento predictivo via técnicas de _machine learning_.

**PREPROCESAMIENTO y RECODIFICACION DE DATOS**

Realizaré el proceso estándar para el preprocesamiento de los datos:

    - revisión del archivo con los datos (encoding, archivo corruptos, etc.);
    - limpieza de valores inválidos si corresponde;
    - recodificación de columnas según lo sugerido y de acorde a las necesidades del problema.

# FIN HITO 1, de aquí en adelante considerar como WIP

In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#import statsmodels.formula.api as smf
#from sklearn.linear_model import LinearRegression
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
#from sklearn.metrics import classification_report, roc_curve, roc_auc_score, confusion_matrix
#plt.style.use('seaborn-notebook')
#plt.rcParams['figure.figsize'] = 10, 6

Primero veamos qué tiene el dataset:

In [2]:
df = pd.read_csv('income-db.csv')
df.sample()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
5410,49,Private,205694,HS-grad,9,Never-married,Other-service,Own-child,White,Female,0,0,40,Canada,<=50K


Transformo los datos `?` a NaN's para poder tratarlos adecuadamente:

In [3]:
df = df.replace(to_replace='?', value=np.nan)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          46043 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         46033 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     47985 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


La recodificación se puede realizar de la siguiente manera. Primero, defino los grupos:

In [5]:
white_collars = ['Prof-specialty', 'Exec-managerial', 'Adm-clerical', 'Sales', 'Tech-support']
blue_collars = ['Craft-repair', 'Machine-op-inspct', 'Transport-moving', 'Handlers-cleaners', 'Farming-fishing', 'Protective-serv', 'Priv-house-serv']
other_collars = ['Other-service', 'Armed-Forces']
federal_gov_wc = ['Federal-gov']
state_level_gov_wc = ['State-goc', 'Local-gov']
self_employed_wc = ['Self-emp-inc', 'Self-emp-not-inc']
unemployed_wc = ['Never-worked', 'Without-pay']
private_wc = ['Private']
preschool_ed = ['Preschool']
elementary_school_ed = ['1st-4th', '5th-6th']
high_school_ed = ['7th-8th', '9th', '10th', '11th', '12th', 'HS-grad']
college_ed = ['Assoc-voc', 'Assoc-acdm', 'Some-college']
university_ed = ['Bachelors', 'Masters', 'Prof-school', 'Doctorate']
married_ms = ['Married-civ-spouse', 'Married-spouse-absent', 'Married-AF-spouse']
divorced_ms = ['Divorced']
separated_ms = ['Separated']
widowed_ms = ['Widowed']
america_reg = ['United-States', 'Mexico', 'Puerto-Rico', 'Canada', 'El-Salvador', 'Cuba', 'Jamaica', 'Dominican-Republic', 'Guatemala', 'Columbia', 'Haiti', 'Nicaragua', 'Peru', 'Ecuador', 'Trinadad&Tobago', 'Outlying-US', 'Honduras']
europe_reg = ['Germany', 'England', 'Italy', 'Poland', 'Portugal', 'Greece', 'France', 'Ireland','Yugoslavia', 'Scotland', 'Hungary', 'Holand-Netherlands']
asia_reg = ['Philippines', 'India', 'China', 'Japan', 'Vietnam', 'Taiwan', 'Iran', 'Hong', 'Thailand', 'Cambodia', 'Laos']
africa_reg = ['South']

In [6]:
def recod_row(row, original, to, labels):
    """For a given dataframe row, creates a new column based on a label list.
    
    Parameters
    ----------
    row : str
        The dataframe row to be based upon
    labels: lst
        List containing all labels
    
    Returns
    -------
    label: str
        Corresponding output label
    """
    return to if row[original] in labels else np.nan

In [7]:
df['collars'] = df.apply(lambda row: recod_row(row, 'occupation', 'white', white_collars), axis=1)
df['collars'] = df.apply(lambda row: recod_row(row, 'occupation', 'blue', blue_collars) if row['collars'] is np.nan else row['collars'], axis=1)
df['collars'] = df.apply(lambda row: recod_row(row, 'occupation', 'others', other_collars) if row['collars'] is np.nan else row['collars'], axis=1)
df['workclass_recod'] = df.apply(lambda row: recod_row(row, 'workclass', 'federal_gov', federal_gov_wc), axis=1)
df['workclass_recod'] = df.apply(lambda row: recod_row(row, 'workclass', 'state_level_gov', state_level_gov_wc) if row['workclass_recod'] is np.nan else row['workclass_recod'], axis=1)
df['workclass_recod'] = df.apply(lambda row: recod_row(row, 'workclass', 'self_employed', self_employed_wc) if row['workclass_recod'] is np.nan else row['workclass_recod'], axis=1)
df['workclass_recod'] = df.apply(lambda row: recod_row(row, 'workclass', 'unemployed', unemployed_wc) if row['workclass_recod'] is np.nan else row['workclass_recod'], axis=1)
df['workclass_recod'] = df.apply(lambda row: recod_row(row, 'workclass', 'private', private_wc) if row['workclass_recod'] is np.nan else row['workclass_recod'], axis=1)
df['educ_recod'] = df.apply(lambda row: recod_row(row, 'education', 'preschool', preschool_ed), axis=1)
df['educ_recod'] = df.apply(lambda row: recod_row(row, 'education', 'elementary_school', elementary_school_ed) if row['educ_recod'] is np.nan else row['educ_recod'], axis=1)
df['educ_recod'] = df.apply(lambda row: recod_row(row, 'education', 'high_school', high_school_ed) if row['educ_recod'] is np.nan else row['educ_recod'], axis=1)
df['educ_recod'] = df.apply(lambda row: recod_row(row, 'education', 'college', college_ed) if row['educ_recod'] is np.nan else row['educ_recod'], axis=1)
df['educ_recod'] = df.apply(lambda row: recod_row(row, 'education', 'university', university_ed) if row['educ_recod'] is np.nan else row['educ_recod'], axis=1)
df['civstatus'] = df.apply(lambda row: recod_row(row, 'marital-status', 'married', married_ms), axis=1)
df['civstatus'] = df.apply(lambda row: recod_row(row, 'marital-status', 'divorced', divorced_ms) if row['civstatus'] is np.nan else row['civstatus'], axis=1)
df['civstatus'] = df.apply(lambda row: recod_row(row, 'marital-status', 'separated', separated_ms) if row['civstatus'] is np.nan else row['civstatus'], axis=1)
df['civstatus'] = df.apply(lambda row: recod_row(row, 'marital-status', 'widowed', widowed_ms) if row['civstatus'] is np.nan else row['civstatus'], axis=1)
df['region'] = df.apply(lambda row: recod_row(row, 'native-country', 'america', america_reg), axis=1)
df['region'] = df.apply(lambda row: recod_row(row, 'native-country', 'europe', europe_reg) if row['region'] is np.nan else row['region'], axis=1)
df['region'] = df.apply(lambda row: recod_row(row, 'native-country', 'asia', asia_reg) if row['region'] is np.nan else row['region'], axis=1)
df['region'] = df.apply(lambda row: recod_row(row, 'native-country', 'africa', africa_reg) if row['region'] is np.nan else row['region'], axis=1)

In [8]:
df['income'].value_counts()

<=50K    37155
>50K     11687
Name: income, dtype: int64

In [9]:
df['>50k'] = np.where(df['income']=='>50K', 1, 0)

Para reducir colinearidad, elimino las columnas que contienen la misma información:

In [10]:
df['>50k'].value_counts()

0    37155
1    11687
Name: >50k, dtype: int64

In [11]:
df = df.drop(columns=['occupation', 'workclass', 'education', 'marital-status', 'native-country'])

In [12]:
df.sample()

,age,fnlwgt,educational-num,relationship,race,gender,capital-gain,capital-loss,hours-per-week,income,collars,workclass_recod,educ_recod,civstatus,region,>50k
31074,44,523484,13,Husband,Black,Male,0,0,40,>50K,white,private,university,married,america,1
